In [1]:
luna_path = '/Volumes/solo/ali/'

In [2]:
import time
import gc
import os

import SimpleITK as sitk
import numpy as np
import csv
import scipy
from glob import glob
import pandas as pd
from scipy import ndimage
from tqdm import tqdm 
import pandas as pd

from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from scipy import ndimage
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import dicom
import scipy.misc
import numpy as np

import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from numba import autojit
import zarr
from PIL import Image
import cv2
import scipy.spatial.distance as dist
import gc
import warnings
from solo.unet_utils import *
from solo.unet_models import *
from solo.preproc_utils import *
from solo.tianchi import *


import tensorflow as tf
from keras import backend as K
K.set_image_dim_ordering('th') 


#from inception_unet import *
#np.random.seed(1337)



os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"


warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [3]:
def unet_fit(name, batch_size, unet_model, load_check = False, check_name = None):
    
    t = time.time()
    callbacks = [EarlyStopping(monitor='val_loss', patience = 5, verbose = 1),
                 ModelCheckpoint((luna_path + 'Data/model/{}.h5').format(name), 
                                 monitor='val_loss', 
                                 verbose = 0,
                                 save_best_only = True)]
    if load_check:
        check_model = (luna_path + 'Data/model/{}.h5').format(check_name)
        model = load_model(check_model, custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})
    else:
        model = unet_model()
        
    model.fit_generator(generate_train(lung_path, nodule_path, start_t, end_t, batch_size), epochs = 4, verbose = 1, 
                        validation_data = generate_val(lung_path, nodule_path, batch_size), callbacks = callbacks, 
                        steps_per_epoch = f1, validation_steps = f2)
    return model

# 训练模型

In [4]:
lung_path = luna_path +  'Data/train_mask/lung_mask/'
nodule_path = luna_path + 'Data/train_mask/nodule_mask/'

start_t, end_t = 0, 500
start_v, end_v = end_t, end_t + 200


f1 = get_max_slices(lung_path, start_t, end_t)
f2 = get_max_slices(lung_path, start_v, end_v)
f1 = f1
f2 = f2//2
print('Training samples:', f1, 'Validation samples:', f2)

smooth = 1.0
img_rows = 512
img_cols = 512
batch_size = 2

model_1 = unet_fit('Unet_fulldatagen428_1', batch_size, unet_model,True,'Unet_fulldatagen428_1')
print 'done!'

Number of 2D slices in CT image: 7697
Number of 2D slices in CT image: 1768
('Training samples:', 7697, 'Validation samples:', 884)
Epoch 1/4
Shuffling data
Shuffling data
7697/7697 [==============================] - 11472s - loss: -1.8736 - dice_coef: 1.8736 - val_loss: -0.8874 - val_dice_coef: 0.8874
Epoch 2/4
7697/7697 [==============================] - 11436s - loss: -1.8587 - dice_coef: 1.8587 - val_loss: -0.8796 - val_dice_coef: 0.8796
Epoch 3/4
7697/7697 [==============================] - 11436s - loss: -1.8676 - dice_coef: 1.8676 - val_loss: -0.7944 - val_dice_coef: 0.7944
Epoch 4/4
7697/7697 [==============================] - 11478s - loss: -1.8724 - dice_coef: 1.8724 - val_loss: -0.9518 - val_dice_coef: 0.9518
done!
